# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Benisty  ->  M. Benisty  |  ['M. Benisty']


L. Acuña  ->  L. Acuña  |  ['L. Acuña']
Arxiv has 67 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2409.17220
extracting tarball to tmp_2409.17220...

 done.


M. Benisty  ->  M. Benisty  |  ['M. Benisty']


Found 95 bibliographic references in tmp_2409.17220/aa.bbl.
Retrieving document from  https://arxiv.org/e-print/2409.18129


extracting tarball to tmp_2409.18129...

 done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2409.18129/aa.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'table_parameters_joint_analysis' from 'tmp_2409.18129/table_parameters_joint_analysis.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'table_HARPS_data' from 'tmp_2409.18129/table_HARPS_data.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Py

Found 235 bibliographic references in tmp_2409.18129/aa.bbl.
Issues with the citations
syntax error in line 450: '}' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.17220-b31b1b.svg)](https://arxiv.org/abs/2409.17220) | **Spatially correlated stellar accretion in the Lupus star forming region: Evidence for ongoing infall from the interstellar medium?**  |
|| A. J. Winter, <mark>M. Benisty</mark>, C. Manara, A. Gupta |
|*Appeared on*| *2024-09-27*|
|*Comments*| *10 pages, 6 figures, accepted for publication in A&A Letters*|
|**Abstract**|            Growing evidence suggests that protoplanetary discs may be influenced by late stage infall from the interstellar medium (ISM). It remains unclear the degree to which infall shapes disc populations at ages $\gtrsim 1$~Myr. We explore possible spatial correlations between stellar accretion rates in the Lupus star forming region, which would support the hypothesis that infall can regulate stellar accretion. We consider both the `clustered' stars towards the center of Lupus 3, and the `distributed' stars that are more sparsely distributed across the Lupus complex. We take the observed accretion rates in the literature and explore spatial correlations. In particular, we test whether the clustered stars exhibit a radial gradient in normalised accretion rates, and whether the distributed stars have spatially correlated accretion rates. We find statistically significant correlations for both the clustered and distributed samples. The clustered sample exhibits higher accretion rates in the central region, consistent with the expected Bondi-Hoyle-Lyttleton accretion rate. Stars that are spatially closer among the distributed population also exhibit more similar accretion rates. These results cannot be explained by the stellar mass distribution for either sample. Age gradients are disfavoured, though not discounted, because normalised disc dust masses are not spatially correlated across the region. Spatially correlated stellar accretion rates within the Lupus star forming region argue in favour of an environmental influence on stellar accretion, possibly combined with internal processes in the inner disc. Refined age measurements and searches for evidence of infalling material is a potential way to further test this finding.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.18129-b31b1b.svg)](https://arxiv.org/abs/2409.18129) | **TOI-5005 b: A super-Neptune in the savanna near the ridge**  |
|| A. Castro-González, et al. -- incl., <mark>L. Acuña</mark> |
|*Appeared on*| *2024-09-27*|
|*Comments*| *Accepted for publication in A&A. Abstract shortened. 35 pages, 26 figures*|
|**Abstract**|            The Neptunian desert and savanna have been recently found to be separated by a ridge, an overdensity of planets in the $\simeq$3-5 days period range. These features are thought to be shaped by dynamical and atmospheric processes. However, their relative roles are not yet well understood. We intend to confirm and characterise the super-Neptune TESS candidate TOI-5005.01, which orbits a moderately bright (V = 11.8) solar-type star (G2 V) with an orbital period of 6.3 days. We confirm TOI-5005 b to be a transiting super-Neptune with a radius of $R_{\rm p}$ = $6.25\pm 0.24$ $\rm R_{\rm \oplus}$ ($R_{\rm p}$ = $0.558\pm 0.021$ $\rm R_{\rm J}$) and a mass of $M_{\rm p}$ = $32.7\pm 5.9$ $\rm M_{\oplus}$ ($M_{\rm p}$ = $0.103\pm 0.018$ $\rm M_{\rm J}$), which corresponds to a mean density of $\rho_{\rm p}$ = $0.74 \pm 0.16$ $\rm g \, cm^{-3}$. Our internal structure modelling indicates that the overall metal mass fraction is well constrained to a value slightly lower than that of Neptune and Uranus ($Z_{\rm planet}$ = $0.76^{+0.04}_{-0.11}$). We also estimated the present-day atmospheric mass-loss rate of TOI-5005 b but found contrasting predictions depending on the choice of photoevaporation model. At a population level, we find statistical evidence ($p$-value = $0.0092^{+0.0184}_{-0.0066}$) that planets in the savanna such as TOI-5005 b tend to show lower densities than planets in the ridge, with a dividing line around 1 $\rm g \, cm^{-3}$, which supports the hypothesis of different evolutionary pathways populating both regimes. TOI-5005 b is located in a key region of the period-radius space to study the transition between the Neptunian ridge and the savanna. It orbits the brightest star of all such planets, which makes it a target of interest for atmospheric and orbital architecture observations that will bring a clearer picture of its overall evolution.         |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2409.17220/./Lupus_100micron.png', 'tmp_2409.17220/./normMdisc_sep.png', 'tmp_2409.17220/./spatial_corr_CDF_Md.png']
copying  tmp_2409.17220/./Lupus_100micron.png to _build/html/
copying  tmp_2409.17220/./normMdisc_sep.png to _build/html/
copying  tmp_2409.17220/./spatial_corr_CDF_Md.png to _build/html/
exported in  _build/html/2409.17220.md
    + _build/html/tmp_2409.17220/./Lupus_100micron.png
    + _build/html/tmp_2409.17220/./normMdisc_sep.png
    + _build/html/tmp_2409.17220/./spatial_corr_CDF_Md.png
found figures ['tmp_2409.18129/./gls_to_HARPS.png', 'tmp_2409.18129/./toi5005.png', 'tmp_2409.18129/./dens_porb.png', 'tmp_2409.18129/./nep_des_TOI-5005.png']
copying  tmp_2409.18129/./gls_to_HARPS.png to _build/html/
copying  tmp_2409.18129/./toi5005.png to _build/html/
copying  tmp_2409.18129/./dens_porb.png to _build/html/
copying  tmp_2409.18129/./nep_des_TOI-5005.png to _build/html/
exported in  _build/html/2409.18129.md
    + _build/html/tmp_2409.18129/./gls_

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Spatially correlated stellar accretion in the Lupus star-forming region: Evidence for ongoing infall from the interstellar medium?

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.17220-b31b1b.svg)](https://arxiv.org/abs/2409.17220)<mark>Appeared on: 2024-09-27</mark> -  _10 pages, 6 figures, accepted for publication in A&A Letters_

</div>
<div id="authors">

A. J. Winter, <mark>M. Benisty</mark>, C. Manara, A. Gupta

</div>
<div id="abstract">

**Abstract:** Growing evidence suggests that protoplanetary discs may be influenced by late stage infall from the interstellar medium (ISM). It remains unclear the degree to which infall shapes disc populations at ages $\gtrsim 1$ Myr. We explored possible spatial correlations between stellar accretion rates in the Lupus star-forming region, which would support the hypothesis that infall can regulate stellar accretion. We considered both the `clustered' stars towards the center of Lupus 3, and the `distributed' stars that are more sparsely distributed across the Lupus complex. We took the observed accretion rates in the literature and explore spatial correlations. In particular, we tested whether the clustered stars exhibit a radial gradient in normalised accretion rates, and whether the distributed stars have spatially correlated accretion rates. We found statistically significant correlations for both the clustered and distributed samples. The clustered sample exhibits higher accretion rates in the central region, consistent with the expected Bondi-Hoyle-Lyttleton accretion rate. Stars that are spatially closer among the distributed population also exhibit more similar accretion rates. These results cannot be explained by the stellar mass distribution for either sample. ${Age gradients are disfavoured, though not discounted,}$ because normalised disc dust masses are not spatially correlated across the region. Spatially correlated stellar accretion rates within the Lupus star-forming region argue in favour of an environmental influence on stellar accretion, possibly combined with internal processes in the inner disc. ${Refined age measurements and}$ searches for evidence of infalling material is a potential way to further test this finding.

</div>

<div id="div_fig1">

<img src="tmp_2409.17220/./Lupus_100micron.png" alt="Fig5" width="100%"/>

**Figure 5. -** {Distribution of candidate Lupus members with _Gaia_ kinematics as identified by [ and Luhman (2020)](), which we show as hollow shapes (fuchsia circles the `on cloud' sample that coincide with Lupus 1-4, and red squares are the `off cloud' sample). The locations of stars with discs included in the PP7 catalogue of [Manara, Ansdell and Rosotti (2023)]() but without measurements of stellar mass, accretion rate or disc dust mass are shown as lime triangles. Those stars which have stellar masses, dust masses and accretion rates recorded by [Manara, Ansdell and Rosotti (2023)]() are shown as filled coloured circles (or downward triangles for upper limits in accretion rate) that represent the normalised accretion rate according to the colour bar. The greyscale overlay is the IRIS $100$ $\mu$m map of the region  ([ and Lagache 2005]()) . We mark with cyan stars and labels the locations of systems which exhibit some evidence of interaction with the ISM or high levels of outer disc turbulence, discussed in Section \ref{sec:individ_syst}. We also show a zoom-in on a region centered on Lupus 3, which we define as the `clustered' region (inside the solid red ellipse). Within this circle, we show the `on cloud' sample with fuchsia circles and inner points to make the sample clear in the crowded central region.} (*fig:lupus*)

</div>
<div id="div_fig2">

<img src="tmp_2409.17220/./normMdisc_sep.png" alt="Fig2" width="100%"/>

**Figure 2. -** As in Figure \ref{subfig:mdacc_sep}, but for normalised disc dust masses rather than accretion rates. (*fig:mdiscsep*)

</div>
<div id="div_fig3">

<img src="tmp_2409.17220/./spatial_corr_CDF_Md.png" alt="Fig3" width="100%"/>

**Figure 3. -** As in Figure \ref{subfig:corr_CDF}, but for normalised disc dust masses rather than accretion rates. (*fig:corr_CDF_Md*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.17220"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\maketitle}{\begingroup\let\footnote=\thanks \ACMmaketitle\endgroup}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.35}$
$\newcommand{\arraystretch}{1.23}$
$\newcommand{\arraystretch}{1.23}$
$\newcommand{\arraystretch}{1.23}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.14}$</div>



<div id="title">

# TOI-5005 b: A super-Neptune in the savanna near the ridge

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.18129-b31b1b.svg)](https://arxiv.org/abs/2409.18129)<mark>Appeared on: 2024-09-27</mark> -  _Accepted for publication in A&A. Abstract shortened. 35 pages, 26 figures_

</div>
<div id="authors">

A. Castro-González, et al. -- incl., <mark>L. Acuña</mark>

</div>
<div id="abstract">

**Abstract:** The Neptunian desert and savanna have been recently found to be separated by a ridge, an overdensity of planets in the $\simeq$ 3-5 days period range. These features are thought to be shaped by dynamical and atmospheric processes. However, their relative roles are not yet well understood. We intend to confirm and characterise the super-Neptune TESS candidate TOI-5005.01, which orbits a moderately bright (V = 11.8) solar-type star (G2 V) with an orbital period of 6.3 days. With such properties, TOI-5005.01 is located in the Neptunian savanna near the ridge. We used Bayesian inference to analyze 38 HARPS radial velocity measurements, three sectors of TESS photometry, and two PEST and TRAPPIST-South transits. We tested a set of models involving eccentric and circular orbits, long-term drifts, and Gaussian processes to account for correlated stellar and instrumental noise. We computed the Bayesian evidences to find the model that best represents our data set and infer the orbital and physical properties of the system. We confirm TOI-5005 b to be a transiting super-Neptune with a radius of $R_{\rm p}$ = $6.25\pm 0.24$  $\rm R_{\rm \oplus}$ ( $R_{\rm p}$ = $0.558\pm 0.021$ $\rm R_{\rm J}$ ) and a mass of $M_{\rm p}$ = $32.7\pm 5.9$ $\rm M_{\oplus}$ ( $M_{\rm p}$ = $0.103\pm 0.018$ $\rm M_{\rm J}$ ), which corresponds to a mean density of $\rho_{\rm p}$ = $0.74 \pm 0.16$ $\rm g   cm^{-3}$ .  Our internal structure modelling indicates that the core mass fraction (CMF = $0.74^{+0.05}_{-0.45}$ ) and envelope metal mass fraction ( $Z_{\rm env}$ = $0.08^{+0.41}_{-0.06}$ ) of TOI-5005 b are degenerate, but the overall metal mass fraction is well constrained to a value slightly lower than that of Neptune and Uranus ( $Z_{\rm planet}$ = $0.76^{+0.04}_{-0.11}$ ). The $Z_{\rm planet}$ / $Z_{\rm star}$ ratio is consistent with the well-known mass-metallicity relation, which suggests that TOI-5005 b was formed via core accretion. We also estimated the present-day atmospheric mass-loss rate of TOI-5005 b but found contrasting predictions depending on the choice of photoevaporation model ( $0.013\pm 0.008$  $\rm M_{\oplus}$ Gyr $^{-1}$ versus $0.17\pm 0.12$  $\rm M_{\oplus}$ Gyr $^{-1}$ ). At a population level, we find statistical evidence ( $p$ -value = $0.0092^{+0.0184}_{-0.0066}$ ) that planets in the savanna such as TOI-5005 b tend to show lower densities than planets in the ridge, with a dividing line around 1 $\rm g   cm^{-3}$ , which supports the hypothesis of different evolutionary pathways populating both regimes. TOI-5005 b is located in a key region of the period-radius space to study the transition between the Neptunian ridge and the savanna. It orbits the brightest star of all such planets known today, which makes it a target of interest for atmospheric and orbital architecture observations that will bring a clearer picture of its overall evolution.

</div>

<div id="div_fig1">

<img src="tmp_2409.18129/./gls_to_HARPS.png" alt="Fig8" width="100%"/>

**Figure 8. -** Left panels: time series of the HARPS RVs, activity indicators, and the window function. The magenta dashed lines represent the linear trends fit to the data. Centre panels: GLS periodograms of the original time series and de-trended time series. The green dotted vertical lines indicate the location of the orbital period of TOI-5005.01 ($P_{\rm orb}$ = 6.3 days). The magenta dotted vertical lines indicate the $\simeq$21 days activity-related signal that most likely reflects the stellar rotation period. The horizontal dotted lines correspond to the 10 (orange), 1 (blue), and 0.1$\%$(green) FAP levels. Right panels: HARPS data folded to the maximum power periods; that is, 6.3 days (RVs), 21.3 days (FWHM), 20.8 days (S-index), 21.0 days (Contrast), 20.5 days (Ca), 6.2 days ($\rm H_{\alpha}$), and 129.8 days (window function). The RV panel shows the de-trended data and the indicators panels show the original time series.  (*fig:gls_to_HARPS*)

</div>
<div id="div_fig2">

<img src="tmp_2409.18129/./toi5005.png" alt="Fig15.1" width="50%"/><img src="tmp_2409.18129/./dens_porb.png" alt="Fig15.2" width="50%"/>

**Figure 15. -** Left: Mass-radius diagram of all known Neptunian planets with masses and radii with precisions better than 20$\%$. Right: Density-period diagram of the same sample. The data were collected from the NASA Exoplanet Archive on 20 September 2024. This plot was generated with \texttt{mr-plotter}\citep[\url{https://github.com/castro-gzlz/mr-plotter};][]{2023A&A...675A..52C}.  (*fig:mass-radius-density-period*)

</div>
<div id="div_fig3">

<img src="tmp_2409.18129/./nep_des_TOI-5005.png" alt="Fig4" width="100%"/>

**Figure 4. -** Contextualization of TOI-5005 b in the period-radius diagram of close-in exoplanets, where we highlight the population-based boundaries of the Neptunian desert, ridge, and savanna derived in \citet{2024A&A...689A.250C}. The data were collected from the NASA Exoplanet Archive \citep{2013PASP..125..989A} on 20 September 2024. This plot has been generated with \texttt{nep-des}(\url{https://github.com/castro-gzlz/nep-des}). (*fig:nep_desert*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.18129"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

199  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
